# Partner Billing Views para Resellers — Versión para **Snowflake Notebooks**
**Fecha:** 2026-01-05
**Autor:** Jonathan Sierra
**Tags:** #snowflake #snowpark #python #sql #billing #partners #resellers #cost

Este cuaderno está optimizado para ejecutarse dentro de **Snowflake Notebooks** usando **Snowpark**.
Cuando se ejecute fuera de Snowflake Notebooks (por ejemplo, en tu Jupyter local), hará *fallback* a una conexión estándar con credenciales.

> **Objetivo:** Habilitar a los Partners y Resellers de Snowflake a tener un control granular del uso y costos de sus clientes mediante las vistas del esquema `BILLING`.

---

## Billing Views for Resellers - Generally Available

Esta funcionalidad permite a los Resale Partners:
- Obtener datos unificados de uso de clientes
- Proporcionar información proactiva y transparente sobre gestión de costos
- Escalar la gestión de costos para todos sus clientes finales

---

## Tabla de Contenidos
1. Parámetros y Conexión
2. Requisitos y Permisos
3. PARTNER_CONTRACT_ITEMS
4. PARTNER_RATE_SHEET_DAILY
5. PARTNER_REMAINING_BALANCE_DAILY
6. PARTNER_USAGE_IN_CURRENCY_DAILY
7. Queries Avanzados de Análisis
8. Reportes y Dashboards


## 1. Parámetros

Ajusta estos valores predeterminados para tu entorno y ventana de análisis.


In [ ]:
# 1.1 Global parameters
M_DAYS = 30
REPORT_DATE = "2025-12-31"  # Fecha para reportes de balance
CUSTOMER_NAME = ""  # Filtro opcional por cliente
CURRENCY = "USD"  # Moneda por defecto


## 2. Conexión y contexto de sesión

Esta celda usa `get_active_session()` dentro de **Snowflake Notebooks**. Si ejecutas localmente, hace fallback para crear una sesión.

**Importante:** Para acceder a las vistas del esquema `BILLING`, necesitas:
- Cuenta de organización o cuenta regular con rol `ORGADMIN` habilitado
- Usar el rol `ACCOUNTADMIN`


In [ ]:
# 2.1 Session and context
from __future__ import annotations
import os, getpass
import pandas as pd
try:
    from snowflake.snowpark.context import get_active_session
    from snowflake.snowpark import Session
    session = get_active_session()
    IN_SF_NB = True
except Exception:
    from snowflake.snowpark import Session
    IN_SF_NB = False

ROLE      = os.getenv("SNOWFLAKE_ROLE",      "ACCOUNTADMIN")
WAREHOUSE = os.getenv("SNOWFLAKE_WAREHOUSE", "COMPUTE_WH")
DATABASE  = os.getenv("SNOWFLAKE_DATABASE",  "SNOWFLAKE")
SCHEMA    = os.getenv("SNOWFLAKE_SCHEMA",    "BILLING")

def get_session():
    global session
    if IN_SF_NB:
        return session
    conn = {
        "account":   os.getenv("SNOWFLAKE_ACCOUNT", "your_account"),
        "user":      os.getenv("SNOWFLAKE_USER", "your_user"),
        "password":  os.getenv("SNOWFLAKE_PASSWORD") or getpass.getpass("Snowflake password: "),
        "role":      ROLE, "warehouse": WAREHOUSE, "database": DATABASE, "schema": SCHEMA
    }
    session = Session.builder.configs(conn).create()
    return session

def run_sql_df(sql: str):
    s = get_session()
    return s.sql(sql).to_pandas()

_ = run_sql_df(f"""
    USE ROLE {ROLE};
    USE WAREHOUSE {WAREHOUSE};
    USE DATABASE {DATABASE};
    USE SCHEMA {SCHEMA};
""")
run_sql_df("SELECT CURRENT_ROLE() ROLE, CURRENT_WAREHOUSE() WH, CURRENT_DATABASE() DB, CURRENT_SCHEMA() SCH;")


---
## 3. Requisitos y Permisos

### Acceso al esquema BILLING

El esquema `SNOWFLAKE.BILLING` contiene vistas de facturación exclusivas para **Resellers y Distribuidores** de Snowflake.

### Requisitos:
1. **Cuenta de Organización** o cuenta regular con `ORGADMIN` habilitado
2. **Rol ACCOUNTADMIN** para acceder a las vistas
3. Latencia de hasta **24 horas** en los datos

### Habilitar el rol ORGADMIN

Si necesitas habilitar el rol ORGADMIN en tu cuenta:


In [ ]:
# 3.1 Verificar si el rol ORGADMIN está disponible
sql = """
SHOW ROLES LIKE 'ORGADMIN';
"""
run_sql_df(sql)


In [ ]:
# 3.2 Habilitar ORGADMIN (ejecutar desde la cuenta de organización)
sql_enable_orgadmin = """
-- Ejecutar desde la cuenta de organización para habilitar ORGADMIN en una cuenta específica
-- SELECT SYSTEM$ENABLE_ORGADMIN_ROLE_FOR_ACCOUNT('<account_locator>');

-- Verificar cuentas en la organización
SELECT * FROM SNOWFLAKE.ORGANIZATION_USAGE.ACCOUNTS;
"""
print("Para habilitar ORGADMIN, ejecutar desde cuenta de organización:")
print(sql_enable_orgadmin)


In [ ]:
# 3.3 Verificar acceso a las vistas de BILLING
sql = """
SHOW VIEWS IN SCHEMA SNOWFLAKE.BILLING;
"""
run_sql_df(sql)


---
## 4. PARTNER_CONTRACT_ITEMS

Esta vista proporciona información sobre los contratos de los clientes del reseller.

### Columnas principales:
| Columna | Descripción |
|---------|-------------|
| ORGANIZATION_NAME | Nombre de la organización del reseller |
| SOLD_TO_ORGANIZATION_NAME | Nombre de la organización del cliente |
| SOLD_TO_CUSTOMER_NAME | Nombre del cliente |
| SOLD_TO_CONTRACT_NUMBER | Número de contrato |
| START_DATE / END_DATE | Fechas del contrato |
| CONTRACT_ITEM | Tipo: capacity, additional capacity, free usage |
| CURRENCY | Moneda del contrato |
| AMOUNT | Monto del contrato |


In [ ]:
# 4.1 Ver todos los contratos de clientes
sql = """
SELECT
    organization_name,
    sold_to_organization_name,
    sold_to_customer_name,
    sold_to_po_number,
    sold_to_contract_number,
    start_date,
    end_date,
    expiration_date,
    contract_item,
    currency,
    amount,
    contract_modified_date
FROM SNOWFLAKE.BILLING.PARTNER_CONTRACT_ITEMS
ORDER BY sold_to_customer_name, start_date DESC;
"""
run_sql_df(sql)


In [ ]:
# 4.2 Resumen de contratos por cliente
sql = """
SELECT
    sold_to_customer_name,
    sold_to_organization_name,
    COUNT(DISTINCT sold_to_contract_number) AS num_contracts,
    SUM(CASE WHEN contract_item = 'capacity' THEN amount ELSE 0 END) AS total_capacity,
    SUM(CASE WHEN contract_item = 'additional capacity' THEN amount ELSE 0 END) AS total_additional_capacity,
    SUM(CASE WHEN contract_item = 'free usage' THEN amount ELSE 0 END) AS total_free_usage,
    SUM(amount) AS total_contract_value,
    MIN(start_date) AS earliest_start,
    MAX(end_date) AS latest_end,
    currency
FROM SNOWFLAKE.BILLING.PARTNER_CONTRACT_ITEMS
GROUP BY sold_to_customer_name, sold_to_organization_name, currency
ORDER BY total_contract_value DESC;
"""
run_sql_df(sql)


In [ ]:
# 4.3 Contratos próximos a vencer (próximos 90 días)
sql = """
SELECT
    sold_to_customer_name,
    sold_to_organization_name,
    sold_to_contract_number,
    contract_item,
    amount,
    currency,
    end_date,
    expiration_date,
    DATEDIFF(day, CURRENT_DATE(), expiration_date) AS days_until_expiration
FROM SNOWFLAKE.BILLING.PARTNER_CONTRACT_ITEMS
WHERE expiration_date BETWEEN CURRENT_DATE() AND DATEADD(day, 90, CURRENT_DATE())
ORDER BY expiration_date ASC;
"""
run_sql_df(sql)


---
## 5. PARTNER_RATE_SHEET_DAILY

Esta vista muestra las tarifas efectivas utilizadas para calcular el uso en la moneda de la organización.

### Columnas principales:
| Columna | Descripción |
|---------|-------------|
| DATE | Fecha (UTC) para el precio efectivo |
| ACCOUNT_NAME / ACCOUNT_LOCATOR | Cuenta del cliente |
| REGION | Región de la cuenta |
| SERVICE_LEVEL | Nivel de servicio (Standard, Enterprise, Business Critical) |
| USAGE_TYPE | Tipo de uso (Compute, Storage, Data Transfer, etc.) |
| BILLING_TYPE | Tipo de cobro (consumption, rebate, priority support, etc.) |
| SERVICE_TYPE | Tipo de servicio (snowpipe, etc.) |
| EFFECTIVE_RATE | Tarifa efectiva después de descuentos |


In [ ]:
# 5.1 Ver tarifas actuales por cliente y tipo de uso
sql = """
SELECT
    sold_to_customer_name,
    account_name,
    region,
    service_level,
    usage_type,
    billing_type,
    service_type,
    currency,
    effective_rate,
    date,
    is_adjustment
FROM SNOWFLAKE.BILLING.PARTNER_RATE_SHEET_DAILY
WHERE date = (SELECT MAX(date) FROM SNOWFLAKE.BILLING.PARTNER_RATE_SHEET_DAILY)
ORDER BY sold_to_customer_name, usage_type;
"""
run_sql_df(sql)


In [ ]:
# 5.2 Comparar tarifas por nivel de servicio
sql = """
SELECT
    service_level,
    usage_type,
    billing_type,
    currency,
    AVG(effective_rate) AS avg_effective_rate,
    MIN(effective_rate) AS min_rate,
    MAX(effective_rate) AS max_rate,
    COUNT(DISTINCT sold_to_customer_name) AS num_customers
FROM SNOWFLAKE.BILLING.PARTNER_RATE_SHEET_DAILY
WHERE date = (SELECT MAX(date) FROM SNOWFLAKE.BILLING.PARTNER_RATE_SHEET_DAILY)
    AND is_adjustment = FALSE
GROUP BY service_level, usage_type, billing_type, currency
ORDER BY service_level, usage_type;
"""
run_sql_df(sql)


In [ ]:
# 5.3 Tarifas por región
sql = """
SELECT
    region,
    service_level,
    usage_type,
    currency,
    AVG(effective_rate) AS avg_effective_rate,
    COUNT(DISTINCT account_name) AS num_accounts
FROM SNOWFLAKE.BILLING.PARTNER_RATE_SHEET_DAILY
WHERE date = (SELECT MAX(date) FROM SNOWFLAKE.BILLING.PARTNER_RATE_SHEET_DAILY)
    AND is_adjustment = FALSE
GROUP BY region, service_level, usage_type, currency
ORDER BY region, service_level, usage_type;
"""
run_sql_df(sql)


---
## 6. PARTNER_REMAINING_BALANCE_DAILY

Esta vista proporciona el saldo restante diario y el consumo on-demand de los clientes.

### Columnas principales:
| Columna | Descripción |
|---------|-------------|
| DATE | Fecha del saldo (fin del día) |
| CAPACITY_BALANCE | Saldo de capacidad disponible |
| FREE_USAGE_BALANCE | Saldo de uso gratuito disponible |
| ON_DEMAND_CONSUMPTION_BALANCE | Consumo on-demand (valor negativo hasta pago) |
| ROLLOVER_BALANCE | Saldo de rollover disponible |
| MARKETPLACE_CAPACITY_DRAWDOWN_BALANCE | Saldo disponible para Marketplace |


In [ ]:
# 6.1 Saldo actual de todos los clientes
sql = """
SELECT
    date AS balance_date,
    sold_to_customer_name,
    sold_to_organization_name,
    currency,
    capacity_balance,
    free_usage_balance,
    rollover_balance,
    on_demand_consumption_balance,
    marketplace_capacity_drawdown_balance,
    capacity_balance + free_usage_balance + rollover_balance AS total_available_balance
FROM SNOWFLAKE.BILLING.PARTNER_REMAINING_BALANCE_DAILY
WHERE date = (SELECT MAX(date) FROM SNOWFLAKE.BILLING.PARTNER_REMAINING_BALANCE_DAILY)
ORDER BY total_available_balance ASC;
"""
run_sql_df(sql)


In [ ]:
# 6.2 Clientes con bajo saldo (menos del 20% de capacidad)
sql = """
WITH current_balance AS (
    SELECT
        sold_to_customer_name,
        sold_to_organization_name,
        currency,
        capacity_balance,
        free_usage_balance,
        rollover_balance,
        capacity_balance + free_usage_balance + rollover_balance AS total_balance
    FROM SNOWFLAKE.BILLING.PARTNER_REMAINING_BALANCE_DAILY
    WHERE date = (SELECT MAX(date) FROM SNOWFLAKE.BILLING.PARTNER_REMAINING_BALANCE_DAILY)
),
contract_amounts AS (
    SELECT
        sold_to_customer_name,
        SUM(amount) AS total_contract_amount
    FROM SNOWFLAKE.BILLING.PARTNER_CONTRACT_ITEMS
    WHERE contract_item IN ('capacity', 'additional capacity')
    GROUP BY sold_to_customer_name
)
SELECT
    cb.sold_to_customer_name,
    cb.sold_to_organization_name,
    cb.total_balance,
    ca.total_contract_amount,
    ROUND(100 * cb.total_balance / NULLIF(ca.total_contract_amount, 0), 2) AS pct_remaining,
    cb.currency,
    CASE
        WHEN cb.total_balance / NULLIF(ca.total_contract_amount, 0) < 0.1 THEN 'CRITICO'
        WHEN cb.total_balance / NULLIF(ca.total_contract_amount, 0) < 0.2 THEN 'BAJO'
        WHEN cb.total_balance / NULLIF(ca.total_contract_amount, 0) < 0.5 THEN 'MODERADO'
        ELSE 'SALUDABLE'
    END AS status
FROM current_balance cb
LEFT JOIN contract_amounts ca ON cb.sold_to_customer_name = ca.sold_to_customer_name
WHERE cb.total_balance / NULLIF(ca.total_contract_amount, 0) < 0.2
ORDER BY pct_remaining ASC;
"""
run_sql_df(sql)


In [ ]:
# 6.3 Tendencia de saldo por cliente (últimos 30 días)
sql = f"""
SELECT
    date,
    sold_to_customer_name,
    capacity_balance,
    free_usage_balance,
    rollover_balance,
    capacity_balance + free_usage_balance + rollover_balance AS total_balance,
    on_demand_consumption_balance
FROM SNOWFLAKE.BILLING.PARTNER_REMAINING_BALANCE_DAILY
WHERE date >= DATEADD(day, -{M_DAYS}, CURRENT_DATE())
ORDER BY sold_to_customer_name, date DESC;
"""
run_sql_df(sql)


In [ ]:
# 6.4 Clientes con consumo on-demand (potencial facturación adicional)
sql = """
SELECT
    date AS balance_date,
    sold_to_customer_name,
    sold_to_organization_name,
    on_demand_consumption_balance,
    currency,
    ABS(on_demand_consumption_balance) AS on_demand_amount_due
FROM SNOWFLAKE.BILLING.PARTNER_REMAINING_BALANCE_DAILY
WHERE date = (SELECT MAX(date) FROM SNOWFLAKE.BILLING.PARTNER_REMAINING_BALANCE_DAILY)
    AND on_demand_consumption_balance < 0
ORDER BY on_demand_consumption_balance ASC;
"""
run_sql_df(sql)


---
## 7. PARTNER_USAGE_IN_CURRENCY_DAILY

Esta vista proporciona el uso diario de créditos y el uso diario en moneda para todos los clientes.

### Columnas principales:
| Columna | Descripción |
|---------|-------------|
| DATE | Fecha del uso |
| ACCOUNT_NAME | Nombre de la cuenta del cliente |
| USAGE_TYPE | Tipo de uso (compute, storage, data_transfer, etc.) |
| SERVICE_TYPE | Tipo de servicio específico |
| BILLING_TYPE | Tipo de facturación |
| CREDITS_USED | Créditos consumidos |
| USAGE_IN_CURRENCY | Uso en moneda local |


In [ ]:
# 7.1 Uso diario por cliente (últimos 30 días)
sql = f"""
SELECT
    date,
    sold_to_customer_name,
    account_name,
    usage_type,
    service_type,
    billing_type,
    credits_used,
    usage_in_currency,
    currency
FROM SNOWFLAKE.BILLING.PARTNER_USAGE_IN_CURRENCY_DAILY
WHERE date >= DATEADD(day, -{M_DAYS}, CURRENT_DATE())
ORDER BY date DESC, sold_to_customer_name, usage_type;
"""
run_sql_df(sql)


In [ ]:
# 7.2 Resumen de uso por cliente (mes actual)
sql = """
SELECT
    sold_to_customer_name,
    sold_to_organization_name,
    COUNT(DISTINCT account_name) AS num_accounts,
    SUM(credits_used) AS total_credits,
    SUM(usage_in_currency) AS total_usage_currency,
    currency,
    SUM(CASE WHEN usage_type = 'compute' THEN usage_in_currency ELSE 0 END) AS compute_cost,
    SUM(CASE WHEN usage_type = 'storage' THEN usage_in_currency ELSE 0 END) AS storage_cost,
    SUM(CASE WHEN usage_type = 'data_transfer' THEN usage_in_currency ELSE 0 END) AS data_transfer_cost
FROM SNOWFLAKE.BILLING.PARTNER_USAGE_IN_CURRENCY_DAILY
WHERE date >= DATE_TRUNC('month', CURRENT_DATE())
GROUP BY sold_to_customer_name, sold_to_organization_name, currency
ORDER BY total_usage_currency DESC;
"""
run_sql_df(sql)


In [ ]:
# 7.3 Uso por tipo de servicio
sql = f"""
SELECT
    sold_to_customer_name,
    usage_type,
    service_type,
    billing_type,
    SUM(credits_used) AS total_credits,
    SUM(usage_in_currency) AS total_cost,
    currency,
    ROUND(100 * RATIO_TO_REPORT(SUM(usage_in_currency)) OVER (PARTITION BY sold_to_customer_name), 2) AS pct_of_customer_total
FROM SNOWFLAKE.BILLING.PARTNER_USAGE_IN_CURRENCY_DAILY
WHERE date >= DATEADD(day, -{M_DAYS}, CURRENT_DATE())
GROUP BY sold_to_customer_name, usage_type, service_type, billing_type, currency
ORDER BY sold_to_customer_name, total_cost DESC;
"""
run_sql_df(sql)


In [ ]:
# 7.4 Tendencia diaria de uso por cliente
sql = f"""
SELECT
    date,
    sold_to_customer_name,
    SUM(credits_used) AS daily_credits,
    SUM(usage_in_currency) AS daily_cost,
    currency
FROM SNOWFLAKE.BILLING.PARTNER_USAGE_IN_CURRENCY_DAILY
WHERE date >= DATEADD(day, -{M_DAYS}, CURRENT_DATE())
GROUP BY date, sold_to_customer_name, currency
ORDER BY sold_to_customer_name, date DESC;
"""
run_sql_df(sql)


In [ ]:
# 7.5 Top 10 cuentas por consumo
sql = f"""
SELECT
    sold_to_customer_name,
    account_name,
    region,
    SUM(credits_used) AS total_credits,
    SUM(usage_in_currency) AS total_cost,
    currency
FROM SNOWFLAKE.BILLING.PARTNER_USAGE_IN_CURRENCY_DAILY
WHERE date >= DATEADD(day, -{M_DAYS}, CURRENT_DATE())
GROUP BY sold_to_customer_name, account_name, region, currency
ORDER BY total_cost DESC
LIMIT 10;
"""
run_sql_df(sql)


---
## 8. Queries Avanzados de Análisis

Queries para análisis más profundo y gestión proactiva de clientes.


### 8.1 Dashboard Ejecutivo - Resumen de todos los clientes


In [ ]:
# 8.1.1 Dashboard ejecutivo completo
sql = """
WITH current_balance AS (
    SELECT
        sold_to_customer_name,
        capacity_balance + free_usage_balance + rollover_balance AS total_balance,
        on_demand_consumption_balance,
        currency
    FROM SNOWFLAKE.BILLING.PARTNER_REMAINING_BALANCE_DAILY
    WHERE date = (SELECT MAX(date) FROM SNOWFLAKE.BILLING.PARTNER_REMAINING_BALANCE_DAILY)
),
contract_info AS (
    SELECT
        sold_to_customer_name,
        SUM(amount) AS total_contract,
        MIN(start_date) AS contract_start,
        MAX(expiration_date) AS contract_end
    FROM SNOWFLAKE.BILLING.PARTNER_CONTRACT_ITEMS
    GROUP BY sold_to_customer_name
),
monthly_usage AS (
    SELECT
        sold_to_customer_name,
        SUM(usage_in_currency) AS mtd_usage,
        SUM(credits_used) AS mtd_credits
    FROM SNOWFLAKE.BILLING.PARTNER_USAGE_IN_CURRENCY_DAILY
    WHERE date >= DATE_TRUNC('month', CURRENT_DATE())
    GROUP BY sold_to_customer_name
)
SELECT
    cb.sold_to_customer_name AS customer,
    cb.currency,
    ci.total_contract,
    cb.total_balance AS remaining_balance,
    ROUND(100 * cb.total_balance / NULLIF(ci.total_contract, 0), 1) AS pct_remaining,
    mu.mtd_usage,
    mu.mtd_credits,
    ci.contract_end,
    DATEDIFF(day, CURRENT_DATE(), ci.contract_end) AS days_until_expiry,
    CASE
        WHEN cb.total_balance / NULLIF(ci.total_contract, 0) < 0.1 THEN 'CRITICO'
        WHEN cb.total_balance / NULLIF(ci.total_contract, 0) < 0.25 THEN 'ALERTA'
        WHEN DATEDIFF(day, CURRENT_DATE(), ci.contract_end) < 60 THEN 'RENOVACION'
        ELSE 'OK'
    END AS status
FROM current_balance cb
LEFT JOIN contract_info ci ON cb.sold_to_customer_name = ci.sold_to_customer_name
LEFT JOIN monthly_usage mu ON cb.sold_to_customer_name = mu.sold_to_customer_name
ORDER BY pct_remaining ASC NULLS LAST;
"""
run_sql_df(sql)


### 8.2 Proyección de agotamiento de créditos


In [ ]:
# 8.2.1 Estimar cuándo se agotarán los créditos de cada cliente
sql = f"""
WITH current_balance AS (
    SELECT
        sold_to_customer_name,
        capacity_balance + free_usage_balance + rollover_balance AS total_balance,
        currency
    FROM SNOWFLAKE.BILLING.PARTNER_REMAINING_BALANCE_DAILY
    WHERE date = (SELECT MAX(date) FROM SNOWFLAKE.BILLING.PARTNER_REMAINING_BALANCE_DAILY)
),
daily_avg_usage AS (
    SELECT
        sold_to_customer_name,
        AVG(daily_usage) AS avg_daily_usage
    FROM (
        SELECT
            sold_to_customer_name,
            date,
            SUM(usage_in_currency) AS daily_usage
        FROM SNOWFLAKE.BILLING.PARTNER_USAGE_IN_CURRENCY_DAILY
        WHERE date >= DATEADD(day, -{M_DAYS}, CURRENT_DATE())
        GROUP BY sold_to_customer_name, date
    )
    GROUP BY sold_to_customer_name
),
contract_end AS (
    SELECT
        sold_to_customer_name,
        MAX(expiration_date) AS contract_expiration
    FROM SNOWFLAKE.BILLING.PARTNER_CONTRACT_ITEMS
    GROUP BY sold_to_customer_name
)
SELECT
    cb.sold_to_customer_name AS customer,
    cb.total_balance,
    cb.currency,
    dau.avg_daily_usage,
    ROUND(cb.total_balance / NULLIF(dau.avg_daily_usage, 0), 0) AS days_until_depletion,
    DATEADD(day, ROUND(cb.total_balance / NULLIF(dau.avg_daily_usage, 0), 0), CURRENT_DATE()) AS projected_depletion_date,
    ce.contract_expiration,
    CASE
        WHEN ROUND(cb.total_balance / NULLIF(dau.avg_daily_usage, 0), 0) < 30 THEN 'URGENTE'
        WHEN ROUND(cb.total_balance / NULLIF(dau.avg_daily_usage, 0), 0) < 60 THEN 'ALERTA'
        WHEN ROUND(cb.total_balance / NULLIF(dau.avg_daily_usage, 0), 0) < 90 THEN 'MONITOREAR'
        ELSE 'OK'
    END AS urgency
FROM current_balance cb
LEFT JOIN daily_avg_usage dau ON cb.sold_to_customer_name = dau.sold_to_customer_name
LEFT JOIN contract_end ce ON cb.sold_to_customer_name = ce.sold_to_customer_name
WHERE dau.avg_daily_usage > 0
ORDER BY days_until_depletion ASC;
"""
run_sql_df(sql)


### 8.3 Análisis de crecimiento de uso por cliente


In [ ]:
# 8.3.1 Comparar uso mes actual vs mes anterior
sql = """
WITH current_month AS (
    SELECT
        sold_to_customer_name,
        SUM(usage_in_currency) AS current_month_usage,
        SUM(credits_used) AS current_month_credits
    FROM SNOWFLAKE.BILLING.PARTNER_USAGE_IN_CURRENCY_DAILY
    WHERE date >= DATE_TRUNC('month', CURRENT_DATE())
    GROUP BY sold_to_customer_name
),
previous_month AS (
    SELECT
        sold_to_customer_name,
        SUM(usage_in_currency) AS previous_month_usage,
        SUM(credits_used) AS previous_month_credits
    FROM SNOWFLAKE.BILLING.PARTNER_USAGE_IN_CURRENCY_DAILY
    WHERE date >= DATE_TRUNC('month', DATEADD('month', -1, CURRENT_DATE()))
      AND date < DATE_TRUNC('month', CURRENT_DATE())
    GROUP BY sold_to_customer_name
)
SELECT
    COALESCE(cm.sold_to_customer_name, pm.sold_to_customer_name) AS customer,
    pm.previous_month_usage,
    cm.current_month_usage,
    cm.current_month_usage - COALESCE(pm.previous_month_usage, 0) AS usage_change,
    ROUND(100 * (cm.current_month_usage - COALESCE(pm.previous_month_usage, 0)) 
        / NULLIF(pm.previous_month_usage, 0), 1) AS pct_change,
    CASE
        WHEN ROUND(100 * (cm.current_month_usage - COALESCE(pm.previous_month_usage, 0)) 
            / NULLIF(pm.previous_month_usage, 0), 1) > 50 THEN 'CRECIMIENTO ALTO'
        WHEN ROUND(100 * (cm.current_month_usage - COALESCE(pm.previous_month_usage, 0)) 
            / NULLIF(pm.previous_month_usage, 0), 1) > 20 THEN 'CRECIMIENTO'
        WHEN ROUND(100 * (cm.current_month_usage - COALESCE(pm.previous_month_usage, 0)) 
            / NULLIF(pm.previous_month_usage, 0), 1) < -20 THEN 'DECREMENTO'
        ELSE 'ESTABLE'
    END AS trend
FROM current_month cm
FULL OUTER JOIN previous_month pm ON cm.sold_to_customer_name = pm.sold_to_customer_name
ORDER BY pct_change DESC NULLS LAST;
"""
run_sql_df(sql)


### 8.4 Detalle por cuenta de un cliente específico


In [ ]:
# 8.4.1 Detalle de uso por cuenta de un cliente (filtrar por CUSTOMER_NAME)
sql = f"""
SELECT
    date,
    account_name,
    account_locator,
    region,
    usage_type,
    service_type,
    credits_used,
    usage_in_currency,
    currency
FROM SNOWFLAKE.BILLING.PARTNER_USAGE_IN_CURRENCY_DAILY
WHERE date >= DATEADD(day, -{M_DAYS}, CURRENT_DATE())
    AND sold_to_customer_name = '{CUSTOMER_NAME if CUSTOMER_NAME else "EXAMPLE_CUSTOMER"}'
ORDER BY date DESC, account_name, usage_type;
"""
if CUSTOMER_NAME:
    run_sql_df(sql)
else:
    print("Define CUSTOMER_NAME en los parámetros para filtrar por cliente específico")
    print(sql)


### 8.5 Reporte mensual consolidado


In [ ]:
# 8.5.1 Reporte mensual para todos los clientes
sql = """
SELECT
    DATE_TRUNC('month', date) AS month,
    sold_to_customer_name,
    sold_to_organization_name,
    COUNT(DISTINCT account_name) AS num_accounts,
    SUM(credits_used) AS total_credits,
    SUM(usage_in_currency) AS total_cost,
    currency,
    SUM(CASE WHEN usage_type = 'compute' THEN usage_in_currency ELSE 0 END) AS compute_cost,
    SUM(CASE WHEN usage_type = 'storage' THEN usage_in_currency ELSE 0 END) AS storage_cost,
    SUM(CASE WHEN usage_type = 'data_transfer' THEN usage_in_currency ELSE 0 END) AS transfer_cost,
    SUM(CASE WHEN usage_type NOT IN ('compute', 'storage', 'data_transfer') 
        THEN usage_in_currency ELSE 0 END) AS other_cost
FROM SNOWFLAKE.BILLING.PARTNER_USAGE_IN_CURRENCY_DAILY
WHERE date >= DATEADD('month', -6, CURRENT_DATE())
GROUP BY DATE_TRUNC('month', date), sold_to_customer_name, sold_to_organization_name, currency
ORDER BY month DESC, total_cost DESC;
"""
run_sql_df(sql)


---
## 9. Referencias y Documentación

### Documentación Oficial de Snowflake:
- [BILLING Schema](https://docs.snowflake.com/en/sql-reference/billing) - Documentación general del esquema BILLING
- [PARTNER_CONTRACT_ITEMS](https://docs.snowflake.com/en/sql-reference/billing/partner_contract_items) - Vista de contratos
- [PARTNER_RATE_SHEET_DAILY](https://docs.snowflake.com/en/sql-reference/billing/partner_rate_sheet_daily) - Vista de tarifas
- [PARTNER_REMAINING_BALANCE_DAILY](https://docs.snowflake.com/en/sql-reference/billing/partner_remaining_balance_daily) - Vista de saldos
- [PARTNER_USAGE_IN_CURRENCY_DAILY](https://docs.snowflake.com/en/sql-reference/billing/partner_usage_in_currency_daily) - Vista de uso diario
- [Organization Accounts](https://docs.snowflake.com/en/user-guide/organization-accounts) - Cuentas de organización
- [Enabling ORGADMIN Role](https://docs.snowflake.com/en/user-guide/organization-administrators#label-enabling-orgadmin-role-for-account) - Habilitar rol ORGADMIN

### Notas Importantes:
- La latencia de las vistas puede ser de hasta **24 horas**
- Hasta el cierre del mes, los datos pueden cambiar debido a ajustes de fin de mes o enmiendas de contrato
- El saldo de consumo on-demand se reinicia después del cierre del mes (típicamente el 3er o 4to día del siguiente mes)
- Requiere rol **ACCOUNTADMIN** para acceder a las vistas
